# 5. Feature Engineering, Feature Selection, and Baseline Benchmark

In [1]:
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import warnings
import numpy as np

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [2]:
#na_filter set to False as otherwise empty strings are interpreted as NaN
df_tweets_cleaned = pd.read_csv('..\data\Tweets_cleaned.csv', encoding='utf-8', na_filter= False)

## Feature Engineering
Let's generate some features we could possibly use. Some features, such as `emojis_flag`, `emoticons_flag`, and `hashtags_flag` are already generated. Below are some of the features we are engineering/generating:

1. `emojis_num` denotes the number of emojis used in a tweet. 
2. `emoitcons_num` denotes the number of emoticons used in a tweet. 
3. `hashtag_num` denotes the number of hashtags used in a tweet.
4. `numbers_flag` denotes whether the tweet contains numbers or not (either in Arabic or English)
5. `numbers_num` denotes the number of times a tweet contains numbers
We noticed that numbers were used in quite a few negative tweets, such as hours, time, dollars, flight numbers, etc. This is why we are generating a binary flag, as well as a numeric count of numbers used in a tweet.
6. `char_length_original` denotes the length of the the user's original tweet. This includes everything (@ mentions, RT retweets, hyperlinks, etc.)
7. `char_length_user` denotes the length of the user's cleaned tweet. The length will be based off the column `text_cleaned`
We also noticed that negative tweets were, on average, longer than positive tweets in terms of character length.
8. `mentions_num` denotes the number of mentions a tweet has (@ mentions)
9. `retweet_flag` denotes if the user's tweet retweeted a tweet (normally the retweet is one of an airline, rarely another user).
No need to create a count for retweets in a user's tweet because it's always 1.
10. `http_flag` denotes if the user's tweet has a HTTP link. 
No need to create a count for http links in a user's tweet because it's always 1 too.

The True/Flase `_flag` will need to be converted into binary flags instead (i.e. True/False into 1/0).

Any of the `_num` columns will likely need to be scaled to a scale from 0 to 1. 

We will also need to vectorize the words in the tweets. To do so, there are several ways of doing so. We could use `word2vec`, `emoji2vec`, or a combination of both of them called `phrase2vec`.

Lastly, we will need to convert airline_sentiment into 0 or 1. In this situation, because we care about classifying negative sentiment tweets, and not really care about whether it's positive or neutral, we decided to group the positive and neutral tweets as `non-negative`. All `non-negative` tweets are class `0`, whereas all `negative` tweets are class `1`.

### Generate columns `emojis_num`, `emoticons_num`, and `hashtag_num`
Generate basic features such as `emojis_num`, `emoticons_num`, `hashtag_num` from already developed columns.

In [3]:
#creates emojis_num column
def create_emojis_num(df):
    df['emojis_num'] = 0 
    
    for i, row in df.iterrows():    
        if df.at[i, 'emojis_flag']:
            tweet_emojis = df.at[i, 'emojis']
            #strip brackets, quote, and spaces
            tweet_emojis_list = list(tweet_emojis.strip('[]').replace("\'", "").strip().split(","))
            emoji_counter = 0
            
            for emoji in tweet_emojis_list:           
                emoji_counter = emoji_counter + 1
            
            df.at[i, 'emojis_num'] = emoji_counter
        else:
            df.at[i, 'emojis_num'] = 0
            
    return df

#creates emoticons_num column
def create_emoticons_num(df):
    df['emoticons_num'] = 0 
    
    for i, row in df.iterrows():    
        if df.at[i, 'emoticons_flag']:
            tweet_emoticons = df.at[i, 'emoticons']
            #strip brackets, quote, and spaces
            tweet_emoticons_list = list(tweet_emoticons.strip('[]').replace("\'", "").strip().split(","))
            emoticons_counter = 0
            
            for emoticon in tweet_emoticons_list:            
                emoticons_counter = emoticons_counter + 1
            
            df.at[i, 'emoticons_num'] = emoticons_counter
        else:
            df.at[i, 'emoticons_num'] = 0
            
    return df

#creates hashtag_num column
def create_hashtags_num (df):
    df['hashtags_num'] = 0 
    
    for i, row in df.iterrows():    
        if df.at[i, 'hashtags_flag']:
            tweet_hashtags = df.at[i, 'hashtags']
            #strip brackets, quote, and spaces
            tweet_hashtags_list = list(tweet_hashtags.strip('[]').replace("\'", "").strip().split(","))
            hashtags_counter = 0
            
            for hashtag in tweet_hashtags_list:            
                hashtags_counter = hashtags_counter + 1
            
            df.at[i, 'hashtags_num'] = hashtags_counter
        else:
            df.at[i, 'hashtags_num'] = 0
            
    return df

In [4]:
df_tweets_cleaned = create_emojis_num(df_tweets_cleaned)
df_tweets_cleaned = create_emoticons_num(df_tweets_cleaned)
df_tweets_cleaned = create_hashtags_num(df_tweets_cleaned)

In [5]:
#df_tweets_cleaned.loc[df_tweets_cleaned['hashtags_flag'] == True]

### Generate columns `numbers_flag`, `numbers_num`
Generate a binary flag and a count of how many times numbers were used in a tweet. Numbers can either be numeric, or in English. English numbers are sometimes considered stop words by Spacy (e.g. "twelve" is a stop word in tweet `568911315026063361`, but "thirty" is not for some reason in tweet `568237684277141504`), and were removed in `lemmas_list`, so we generate the numbers features from column `text_cleaned_no_abbreviations`. We will use Spacy model to help us determine which token are like numbers, using `like_num`.

In [6]:
df_tweets_cleaned.loc[df_tweets_cleaned['tweet_id'] == 568911315026063361]

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,text,text_cleaned,text_cleaned_time_removed,emojis_flag,emojis,emoticons_flag,emoticons,text_cleaned_without_emojis_emoticons,hashtags,text_cleaned_without_emojis_emoticons_hashtags,hashtags_flag,text_cleaned_lower_case,text_cleaned_no_abbreviations,text_list_no_stop_words,lemmas_list,emojis_num,emoticons_num,hashtags_num
2767,568911315026063361,negative,1.0,Late Flight,1.0,United,@united 5.5 hours Late Flightr I've been in tr...,5.5 hours Late Flightr I've been in transit f...,5.5 hours Late Flightr I've been in transit f...,False,[],False,[],5.5 hours Late Flightr I've been in transit f...,[],5.5 hours Late Flightr I've been in transit f...,False,5.5 hours late flightr i've been in transit f...,5.5 hours late flightr i've been in transit f...,hours late flightr transit total hours change ...,hour late flightr transit total hour change pl...,0,0,0


In [9]:
#load spacy model
import spacy

nlp = spacy.load('en_core_web_md')

In [10]:
#this function will create the columns numbers_flag and numbers_num
def create_numbers_columns(df):
    df['numbers_flag'] = False
    df['numbers_num'] = 0
    
    for i, row in df.iterrows():   
        if i % 1000 == 0:
            print('at row number: ' + str(i))

        text = df.at[i, 'text_cleaned_no_abbreviations']
        #print(type(text))

        like_num_count = 0
        
        #tokenize text into list of tokens
        #print(text)
        
        token_list = nlp(text)

        #iterate through our tokens and count the number of nums
        for token in token_list:
            #print(token)
            if token.like_num:
                like_num_count = like_num_count + 1

        #at the end, we set our new columns
        if like_num_count != 0:
            df.at[i, 'numbers_flag'] = True
            df.at[i, 'numbers_num'] = like_num_count            
    
    return df

In [11]:
#Sanity check

create_numbers_columns(df_tweets_cleaned.loc[df_tweets_cleaned['tweet_id'] == 568911315026063361])
#create_numbers_columns(df_tweets_cleaned.loc[df_tweets_cleaned['tweet_id'] == 570093964059156481])

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,text,text_cleaned,text_cleaned_time_removed,emojis_flag,emojis,emoticons_flag,emoticons,text_cleaned_without_emojis_emoticons,hashtags,text_cleaned_without_emojis_emoticons_hashtags,hashtags_flag,text_cleaned_lower_case,text_cleaned_no_abbreviations,text_list_no_stop_words,lemmas_list,emojis_num,emoticons_num,hashtags_num,numbers_flag,numbers_num
2767,568911315026063361,negative,1.0,Late Flight,1.0,United,@united 5.5 hours Late Flightr I've been in tr...,5.5 hours Late Flightr I've been in transit f...,5.5 hours Late Flightr I've been in transit f...,False,[],False,[],5.5 hours Late Flightr I've been in transit f...,[],5.5 hours Late Flightr I've been in transit f...,False,5.5 hours late flightr i've been in transit f...,5.5 hours late flightr i've been in transit f...,hours late flightr transit total hours change ...,hour late flightr transit total hour change pl...,0,0,0,True,3


In [12]:
df_tweets_cleaned = create_numbers_columns(df_tweets_cleaned)

at row number: 0
at row number: 1000
at row number: 2000
at row number: 3000
at row number: 4000
at row number: 5000
at row number: 6000
at row number: 7000
at row number: 8000
at row number: 9000
at row number: 10000
at row number: 11000
at row number: 12000
at row number: 13000
at row number: 14000


In [13]:
#df_tweets_cleaned.loc[df_tweets_cleaned['numbers_flag'] == True]

### Generate columns `char_length_original`, `char_length_user`
Generate columns with the number of characters in original tweet, and cleaned tweet from column `text_cleaned`.

In [14]:
#this function will create the columns numbers_flag and numbers_num
def create_char_length_columns(df):
    df['char_length_original'] = 0
    df['char_length_user'] = 0
    
    for i, row in df.iterrows():
        text = df.at[i, 'text']
        cleaned_text = df.at[i, 'text_cleaned_no_abbreviations']
        
        df.at[i, 'char_length_original'] = len(text)
        df.at[i, 'char_length_user'] = len(cleaned_text)
    
    return df

In [15]:
df_tweets_cleaned = create_char_length_columns(df_tweets_cleaned)

### Generate columns `mentions_num`, `retweet_flag`, and `http_flag`
Generate columns `mentions_num`: number of mentions in a tweet, `retweet_flag`: whether a tweet has a retweet, and `http_flag`: whether a tweet has a http link.

In [16]:
import re

#this function will create mentions_num column
def create_mentions_num(df):
    df['mentions_num'] = 0
    
    for i, row in df.iterrows():
        text = df.at[i, 'text']
        regex_to_find = r'\@[\w\d]*'
        
        regex_hits_list = re.findall(regex_to_find, text)
        df.at[i, 'mentions_num'] = len(regex_hits_list)
    
    return df

#this function will create retweet_flag column
def create_retweet_flag(df):
    df['retweet_flag'] = False
    
    for i, row in df.iterrows():
        text = df.at[i, 'text']
        regex_to_find = r'RT \@.*'
        
        regex_hits_list = re.findall(regex_to_find, text)
        if (len(regex_hits_list) != 0):
            df.at[i, 'retweet_flag'] = True
    
    return df

#this function will create http_flag column
def create_http_flag(df):
    df['http_flag'] = False
    
    for i, row in df.iterrows():
        text = df.at[i, 'text']
        regex_to_find = r'https*://[^\s]*'
        
        regex_hits_list = re.findall(regex_to_find, text)
        if (len(regex_hits_list) != 0):
            df.at[i, 'http_flag'] = True
    
    return df

In [17]:
df_tweets_cleaned = create_mentions_num(df_tweets_cleaned)
df_tweets_cleaned = create_retweet_flag(df_tweets_cleaned)
df_tweets_cleaned = create_http_flag(df_tweets_cleaned)

### Scale numeric columns
The numeric columns will likely need to be scaled to a scale from 0 to 1. For columns `char_length_original` and `char_length_user` we will use normal MinMaxScaler because there aren't any big outliers, but for the other columns we will use RobustScaler as there are outliers.

In [18]:
df_tweets_cleaned[['emojis_num', 'emoticons_num', 'hashtags_num', 'numbers_num', 'char_length_original', 'char_length_user', 'mentions_num']].describe()

,emojis_num,emoticons_num,hashtags_num,numbers_num,char_length_original,char_length_user,mentions_num
count,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000,14640.000000
mean,0.066667,0.019262,0.238525,0.426571,103.836066,88.200068,1.132719
std,0.612111,0.140400,0.654195,0.738649,36.281706,36.835608,0.410359
min,0.000000,0.000000,0.000000,0.000000,12.000000,0.000000,1.000000
25%,0.000000,0.000000,0.000000,0.000000,77.000000,59.000000,1.000000
50%,0.000000,0.000000,0.000000,0.000000,114.000000,96.000000,1.000000
75%,0.000000,0.000000,0.000000,1.000000,136.000000,121.000000,1.000000
max,40.000000,3.000000,8.000000,7.000000,186.000000,177.000000,6.000000


In [19]:
from sklearn.preprocessing import MinMaxScaler, RobustScaler

minMaxScaler = MinMaxScaler()
df_tweets_cleaned['char_length_original_scaled'] = 0
df_tweets_cleaned['char_length_user_scaled'] = 0
df_tweets_cleaned[['char_length_original_scaled', 'char_length_user_scaled']] = \
    minMaxScaler.fit_transform(df_tweets_cleaned[['char_length_original', 'char_length_user']])

robustScaler = RobustScaler()
df_tweets_cleaned['emojis_num_scaled'] = 0
df_tweets_cleaned['emoticons_num_scaled'] = 0
df_tweets_cleaned['hashtags_num_scaled'] = 0
df_tweets_cleaned['numbers_num_scaled'] = 0
df_tweets_cleaned['mentions_num_scaled'] = 0
df_tweets_cleaned[['emojis_num_scaled', 'emoticons_num_scaled', 'hashtags_num_scaled', 'numbers_num_scaled', 'mentions_num_scaled']] = \
    minMaxScaler.fit_transform(df_tweets_cleaned[['emojis_num', 'emoticons_num', 'hashtags_num', 'numbers_num', 'mentions_num']])

### Convert binary True/False columns to 1s/0s
We should convert binary True/False columns to 1s/0s.

In [20]:
df_tweets_cleaned['emojis_flag'] = df_tweets_cleaned['emojis_flag'].astype(int)
df_tweets_cleaned['emoticons_flag'] = df_tweets_cleaned['emoticons_flag'].astype(int)
df_tweets_cleaned['hashtags_flag'] = df_tweets_cleaned['hashtags_flag'].astype(int)
df_tweets_cleaned['numbers_flag'] = df_tweets_cleaned['numbers_flag'].astype(int)
df_tweets_cleaned['retweet_flag'] = df_tweets_cleaned['retweet_flag'].astype(int)
df_tweets_cleaned['http_flag'] = df_tweets_cleaned['http_flag'].astype(int)

### Group the positive and neutral 
As stated before, our goal is to predict negative sentiment tweets; we don't particularly care if the tweets are positive or neutral, to us they are the same thing: not negative. Therefore, we merge the positive and neutral classes to 0s, and rename negative label to 1s.

In [21]:
df_tweets_cleaned['binary_response_variable'] = False

df_tweets_cleaned.loc[df_tweets_cleaned.airline_sentiment == 'neutral', 'binary_response_variable'] = False
df_tweets_cleaned.loc[df_tweets_cleaned.airline_sentiment == 'positive', 'binary_response_variable'] = False
df_tweets_cleaned.loc[df_tweets_cleaned.airline_sentiment == 'negative', 'binary_response_variable'] = True

In [22]:
df_tweets_cleaned.binary_response_variable

0        False
1        False
2        False
3         True
4         True
         ...  
14635    False
14636     True
14637    False
14638     True
14639    False
Name: binary_response_variable, Length: 14640, dtype: bool

In [23]:
df_tweets_cleaned.binary_response_variable = df_tweets_cleaned.binary_response_variable.astype(int)

In [24]:
df_tweets_cleaned

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,text,text_cleaned,text_cleaned_time_removed,emojis_flag,emojis,emoticons_flag,emoticons,text_cleaned_without_emojis_emoticons,hashtags,text_cleaned_without_emojis_emoticons_hashtags,hashtags_flag,text_cleaned_lower_case,text_cleaned_no_abbreviations,text_list_no_stop_words,lemmas_list,emojis_num,emoticons_num,hashtags_num,numbers_flag,numbers_num,char_length_original,char_length_user,mentions_num,retweet_flag,http_flag,char_length_original_scaled,char_length_user_scaled,emojis_num_scaled,emoticons_num_scaled,hashtags_num_scaled,numbers_num_scaled,mentions_num_scaled,binary_response_variable
0,570306133677760513,neutral,1.0000,,,Virgin America,@VirginAmerica What @dhepburn said.,What said.,What said.,0,[],0,[],What said.,[],What said.,0,what said.,what said.,said,say,0,0,0,0,0,35,12,2,0,0,0.132184,0.067797,0.0,0.0,0.000,0.000000,0.2,0
1,570301130888122368,positive,0.3486,,0.0,Virgin America,@VirginAmerica plus you've added commercials t...,plus you've added commercials to the experien...,plus you've added commercials to the experien...,0,[],0,[],plus you've added commercials to the experien...,[],plus you've added commercials to the experien...,0,plus you've added commercials to the experien...,plus you've added commercials to the experien...,plus added commercials experience tacky,plus add commercial experience tacky,0,0,0,0,0,72,58,1,0,0,0.344828,0.327684,0.0,0.0,0.000,0.000000,0.0,0
2,570301083672813571,neutral,0.6837,,,Virgin America,@VirginAmerica I didn't today... Must mean I n...,I didn't today... Must mean I need to take an...,I didn't today... Must mean I need to take an...,0,[],0,[],I didn't today... Must mean I need to take an...,[],I didn't today... Must mean I need to take an...,0,i didn't today... must mean i need to take an...,i didn't today... must mean i need to take an...,today mean need trip,today mean need trip,0,0,0,0,0,71,57,1,0,0,0.339080,0.322034,0.0,0.0,0.000,0.000000,0.0,0
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,@VirginAmerica it's really aggressive to blast...,"it's really aggressive to blast obnoxious ""en...","it's really aggressive to blast obnoxious ""en...",0,[],0,[],"it's really aggressive to blast obnoxious ""en...",[],"it's really aggressive to blast obnoxious ""en...",0,"it's really aggressive to blast obnoxious ""en...","it's really aggressive to blast obnoxious ""en...",aggressive blast obnoxious entertainment guest...,aggressive blast obnoxious entertainment guest...,0,0,0,0,0,126,108,1,0,0,0.655172,0.610169,0.0,0.0,0.000,0.000000,0.0,1
4,570300817074462722,negative,1.0000,Can't Tell,1.0,Virgin America,@VirginAmerica and it's a really big bad thing...,and it's a really big bad thing about it,and it's a really big bad thing about it,0,[],0,[],and it's a really big bad thing about it,[],and it's a really big bad thing about it,0,and it's a really big bad thing about it,and it's a really big bad thing about it,big bad thing,big bad thing,0,0,0,0,0,55,41,1,0,0,0.247126,0.231638,0.0,0.0,0.000,0.000000,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14635,569587686496825344,positive,0.3487,,0.0,American,@AmericanAir thank you we got on a different f...,thank you we got on a different flight to Chi...,thank you we got on a different flight to Chi...,0,[],0,[],thank you we got on a different flight to Chi...,[],thank you we got on a different flight to Chi...,0,thank you we got on a different flight to chi...,thank you we got on a different flight to chi...,thank got different flight chicago,thank get different flight chicago,0,0,0,0,0,63,51,1,0,0,0.293103,0.288136,0.0,0.0,0.000,0.000000,0.0,0
14636,569587371693355008,negative,1.0000,Customer Service Issue,1.0,American,@AmericanAir leaving over 20 minutes Late Flig...,leaving over 20 minutes Late F

### Get rid of all other columns
We only need the `binary_response_variable`, `_flag` columns, `_scaled` columns, and `lemmas_list` column (this will be vectorized using our models).

In [25]:
df_tweets_cleaned.columns

Index(['tweet_id', 'airline_sentiment', 'airline_sentiment_confidence',
       'negativereason', 'negativereason_confidence', 'airline', 'text',
       'text_cleaned', 'text_cleaned_time_removed', 'emojis_flag', 'emojis',
       'emoticons_flag', 'emoticons', 'text_cleaned_without_emojis_emoticons',
       'hashtags', 'text_cleaned_without_emojis_emoticons_hashtags',
       'hashtags_flag', 'text_cleaned_lower_case',
       'text_cleaned_no_abbreviations', 'text_list_no_stop_words',
       'lemmas_list', 'emojis_num', 'emoticons_num', 'hashtags_num',
       'numbers_flag', 'numbers_num', 'char_length_original',
       'char_length_user', 'mentions_num', 'retweet_flag', 'http_flag',
       'char_length_original_scaled', 'char_length_user_scaled',
       'emojis_num_scaled', 'emoticons_num_scaled', 'hashtags_num_scaled',
       'numbers_num_scaled', 'mentions_num_scaled',
       'binary_response_variable'],
      dtype='object')

In [26]:
df_tweets_cleaned = df_tweets_cleaned[[
    'binary_response_variable', 
    'emojis_flag', 
    'emoticons_flag',
    'hashtags_flag', 
    'numbers_flag',
    'retweet_flag',
    'http_flag',
    'char_length_original_scaled',
    'char_length_user_scaled',
    'emojis_num_scaled',
    'emoticons_num_scaled',
    'hashtags_num_scaled',
    'numbers_num_scaled',
    'mentions_num_scaled',
    'lemmas_list'
]]

## Baseline Benchmark with trained word2vec gensim model
We do a baseline benchmark with Patrick's trained word2vec gensim model, located at `..\milestone1\Patrick\gensim_word2vec_trained.bin` of this repo.

### Split into train and test datasets
We need to split our dataframe into train and test datasets/dataframes. We don't need a validation set for now, but we will split into one later for our machine learning model tuning. For now, 70%/30% for train/test seems good enough. We will split based on column `binary_response_variable`.

In [25]:
from sklearn.model_selection import train_test_split

X = df_tweets_cleaned.loc[:, df_tweets_cleaned.columns != 'binary_response_variable']
Y = df_tweets_cleaned.loc[:, df_tweets_cleaned.columns == 'binary_response_variable']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1, stratify=Y)

### Generate Tweet vectors

In [26]:
import gensim

custom_w2v_gensim_model_path = '..\milestone1\Patrick\gensim_word2vec_trained.bin'
w2v_gensim_model = gensim.models.KeyedVectors.load_word2vec_format(custom_w2v_gensim_model_path, binary=True)

In [27]:
#calculates a vector for a given Tweet
def calculate_average_tweet_vector(tweet, w2v_model, num_dimensions):
    tokens = tweet.split(' ')
    
    tweet_vector = np.zeros(num_dimensions, np.float32)
    actual_token_count = 0
    
    for token in  tokens:
        if token in w2v_model.wv.vocab:
            actual_token_count = actual_token_count + 1
            tweet_vector = np.add(tweet_vector, w2v_model[token])
        
    tweet_vector = np.divide(tweet_vector, actual_token_count)
    
    return tweet_vector

In [28]:
#Sanity check
#calculate_average_tweet_vector("arrangement reimburse rental", w2v_gensim_model, w2v_gensim_model.wv.vector_size)

In [29]:
num_dimensions = w2v_gensim_model.wv.vector_size
X_train['tweet_vector'] = X_train.lemmas_list.apply(lambda text: calculate_average_tweet_vector(text, w2v_gensim_model, num_dimensions))
X_test['tweet_vector'] = X_test.lemmas_list.apply(lambda text: calculate_average_tweet_vector(text, w2v_gensim_model, num_dimensions))

In [30]:
df_train_tweet_vector = pd.DataFrame(list(X_train['tweet_vector']), index=X_train.index)
df_test_tweet_vector = pd.DataFrame(list(X_test['tweet_vector']), index=X_test.index)

In [31]:
#after creating new dataframes above, any zeroes are converted to null...we need to convert them back to zeroes...
df_train_tweet_vector = df_train_tweet_vector.fillna(0)
df_test_tweet_vector = df_test_tweet_vector.fillna(0)

In [32]:
X_train_combined = pd.concat([df_train_tweet_vector, X_train], axis=1)
X_test_combined = pd.concat([df_test_tweet_vector, X_test], axis=1)

### Remove `tweet_vector` and `lemmas_list` column

In [33]:
X_train_combined = X_train_combined.loc[:, X_train_combined.columns != 'tweet_vector']
X_test_combined = X_test_combined.loc[:, X_test_combined.columns != 'tweet_vector']
X_train_combined = X_train_combined.loc[:, X_train_combined.columns != 'lemmas_list']
X_test_combined = X_test_combined.loc[:, X_test_combined.columns != 'lemmas_list']

In [34]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(C=100)
logmodel.fit(X_train_combined, Y_train)
predictions = logmodel.predict(X_test_combined)

In [35]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,predictions))

              precision    recall  f1-score   support

           0       0.79      0.71      0.75      1639
           1       0.84      0.89      0.86      2753

   micro avg       0.82      0.82      0.82      4392
   macro avg       0.81      0.80      0.81      4392
weighted avg       0.82      0.82      0.82      4392



## Baseline Benchmark with Google word2vec gensim model subset
We do a baseline benchmark with Google's word2vec gensim model that is subset with our word vectors as well, located at `..\milestone1\Patrick\gensim_word2vec_google_subset.bin`, which can be unzipped from `..\milestone1\Patrick\gensim_word2vec_google_subset.bin` of this repo.

### Split into train and test datasets
We need to split our dataframe into train and test datasets/dataframes. We don't need a validation set for now, but we will split into one later for our machine learning model tuning. For now, 70%/30% for train/test seems good enough. We will split based on column `binary_response_variable`.

In [36]:
from sklearn.model_selection import train_test_split

X = df_tweets_cleaned.loc[:, df_tweets_cleaned.columns != 'binary_response_variable']
Y = df_tweets_cleaned.loc[:, df_tweets_cleaned.columns == 'binary_response_variable']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1, stratify=Y)

### Generate Tweet vectors

In [37]:
import gensim

custom_w2v_gensim_model_path = '..\milestone1\Patrick\gensim_word2vec_google_subset.bin'
w2v_gensim_model = gensim.models.KeyedVectors.load_word2vec_format(custom_w2v_gensim_model_path, binary=True)

In [136]:
#calculates a vector for a given Tweet
def calculate_average_tweet_vector(tweet, w2v_model, num_dimensions):
    tokens = tweet.split(' ')
    
    tweet_vector = np.zeros(num_dimensions, np.float32)
    actual_token_count = 0
    
    for token in  tokens:
        if token in w2v_model.wv.vocab:
            actual_token_count = actual_token_count + 1
            tweet_vector = np.add(tweet_vector, w2v_model[token])
        
    tweet_vector = np.divide(tweet_vector, actual_token_count)
    
    return tweet_vector

In [137]:
#Sanity check
#calculate_average_tweet_vector("arrangement reimburse rental", w2v_gensim_model, w2v_gensim_model.wv.vector_size)

In [40]:
num_dimensions = w2v_gensim_model.wv.vector_size
X_train['tweet_vector'] = X_train.lemmas_list.apply(lambda text: calculate_average_tweet_vector(text, w2v_gensim_model, num_dimensions))
X_test['tweet_vector'] = X_test.lemmas_list.apply(lambda text: calculate_average_tweet_vector(text, w2v_gensim_model, num_dimensions))

In [41]:
df_train_tweet_vector = pd.DataFrame(list(X_train['tweet_vector']), index=X_train.index)
df_test_tweet_vector = pd.DataFrame(list(X_test['tweet_vector']), index=X_test.index)

In [42]:
#after creating new dataframes above, any zeroes are converted to null...we need to convert them back to zeroes...
df_train_tweet_vector = df_train_tweet_vector.fillna(0)
df_test_tweet_vector = df_test_tweet_vector.fillna(0)

In [43]:
X_train_combined = pd.concat([df_train_tweet_vector, X_train], axis=1)
X_test_combined = pd.concat([df_test_tweet_vector, X_test], axis=1)

### Remove `tweet_vector` and `lemmas_list` column

In [44]:
X_train_combined = X_train_combined.loc[:, X_train_combined.columns != 'tweet_vector']
X_test_combined = X_test_combined.loc[:, X_test_combined.columns != 'tweet_vector']
X_train_combined = X_train_combined.loc[:, X_train_combined.columns != 'lemmas_list']
X_test_combined = X_test_combined.loc[:, X_test_combined.columns != 'lemmas_list']

In [45]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(C=100)
logmodel.fit(X_train_combined, Y_train)
predictions = logmodel.predict(X_test_combined)

In [46]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,predictions))

              precision    recall  f1-score   support

           0       0.79      0.70      0.74      1639
           1       0.83      0.89      0.86      2753

   micro avg       0.82      0.82      0.82      4392
   macro avg       0.81      0.80      0.80      4392
weighted avg       0.82      0.82      0.82      4392



In [47]:
predictions

array([1, 1, 1, ..., 0, 1, 0])

## Baseline Benchmark with GloVe Word Embeddings
We do a baseline benchmark with a GloVe word embedding built with the en_vectors_web_lg model from https://spacy.io/models/en-starters#en_vectors_web_lg.

### Split into train and test datasets
We need to split our dataframe into train and test datasets/dataframes. We don't need a validation set for now, but we will split into one later for our machine learning model tuning. For now, 70%/30% for train/test seems good enough. We will split based on column `binary_response_variable`.

In [27]:
from sklearn.model_selection import train_test_split

X = df_tweets_cleaned.loc[:, df_tweets_cleaned.columns != 'binary_response_variable']
Y = df_tweets_cleaned.loc[:, df_tweets_cleaned.columns == 'binary_response_variable']

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=1, stratify=Y)

### Load up GloVe embeddings for our data set

In [102]:
glove_embed = pd.read_csv('..\milestone1\Patrick\glove_embeddings.csv', encoding='utf-8', na_filter= False)

glove_embed = glove_embed.set_index(['word'])
glove_embed
#glove_embed.drop(['word'], axis='columns')

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
word,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
kickin,0.075017,-0.023001,0.051686,-0.367610,0.43434,0.433910,-0.093183,-0.159130,-0.027413,-0.11179,0.20615,-0.020298,0.082501,0.12975,0.41966,-0.041732,0.165210,-0.52090,-0.179810,0.35513,0.083823,0.430550,0.150440,0.127590,-0.35571,-0.562830,-0.24311,-0.063661,0.670610,-0.680890,0.28912,0.133630,0.009591,-0.063633,-0.008349,0.029259,0.018638,0.516970,0.195890,0.228500,-0.531950,0.048137,-0.31520,0.005763,-0.053208,0.349930,0.37639,-0.315760,-0.374280,0.052387,0.185770,0.450870,0.131280,-0.117590,0.009999,0.117730,-0.117560,-0.198070,0.026480,-0.118760,-0.26134,-0.177580,-0.005444,0.100560,0.049649,-0.368920,-0.135560,0.016130,0.386310,-0.11343,-0.122690,0.105560,0.19637,0.229590,-0.237520,-0.671600,0.361180,0.21771,-0.402070,-0.089469,0.180330,0.077790,0.070386,-0.018462,-0.363890,0.411430,0.938090,-0.13455,-0.03043,-0.032169,0.056822,0.099373,-0.413850,0.548230,-0.025115,0.13147,0.14710,-0.252350,0.277850,0.059278,0.11406,0.113230,-0.168430,-0.204820,0.023099,-0.39080,0.788570,0.055729,0.419220,0.021714,-0.142980,-0.14682,-0.211830,0.088033,0.207400,0.45125,0.265880,-0.087212,-0.273970,-0.13372,-0.19290,-0.229510,-0.230000,-0.047215,0.12610,0.271990,-0.200930,-0.220840,0.190330,-0.066211,0.008174,0.20859,-0.62571,-0.172680,0.211370,0.230690,-0.246950,0.310910,-0.263580,0.06417,-2.32810,-0.117100,0.432230,-0.067614,-0.148330,0.224110,0.039364,0.175830,0.52134,0.178660,-0.555620,-0.148810,0.250330,-0.331600,-0.33799,-0.10787,0.038649,-0.465530,0.019682,-0.257740,-0.093915,0.131480,0.089667,0.591980,0.089008,-0.213290,0.333700,0.12154,-0.135710,-0.082784,0.186070,0.089097,0.18277,-0.179380,-0.244060,-0.262390,-0.30790,0.059947,-0.437410,0.078311,-0.074773,0.035496,0.129550,0.693220,0.089571,0.097667,-0.33291,0.014991,0.493410,0.181180,0.16715,-0.283100,-0.36652,0.050067,-0.030613,0.553300,-0.219470,-0.421560,0.018422,-0.440590,-0.266270,-0.300950,0.339790,0.28471,0.074134,-0.454750,0.381670,0.222230,0.248380,-0.50379,-0.69375,0.33271,0.210850,0.46026,-0.49287,0.54486,0.072352,-0.007455,0.317720,-0.273400,-0.35851,-0.098893,-0.405100,-0.50616,0.310940,-0.348170,-0.494390,0.060978,0.61786,-0.179150,-0.168860,0.223490,-0.490580,-0.65309,0.418200,0.404160,-0.184830,-0.291650,0.277390,0.064184,-0.166560,0.206290,0.045609,0.390090,-0.415420,0.20624,0.058094,-0.21603,0.379090,0.29484,0.026222,-0.423490,0.306030,0.669580,0.103700,-0.43903,-0.21032,-0.313650,-0.891660,0.110780,-0.595340,0.068521,-0.430840,-0.227560,-0.169380,-0.070666,-0.191420,0.176450,-0.094521,-0.067516,0.028811,-0.112830,-0.802890,-0.241350,0.24701,-0.41415,0.004753,-0.201360,0.099871,-0.203710,

### Generate Tweet vectors

In [28]:
doc_glove_vectors = np.array([nlp(str(doc)).vector for doc in df_tweets_cleaned['lemmas_list'].values.astype('U')])

In [33]:
doc_glove_vectors

array([[-0.32077   ,  0.51624   , -0.35279   , ...,  0.18402   ,
         0.45181   ,  0.052621  ],
       [-0.0870746 ,  0.0595982 , -0.041057  , ..., -0.41754597,
         0.12342861,  0.2258898 ],
       [ 0.00190499,  0.1551125 , -0.13563375, ..., -0.058292  ,
         0.02906125, -0.09339049],
       ...,
       [-0.15664567,  0.05659134,  0.3048843 , ..., -0.04364666,
         0.07198433,  0.295372  ],
       [-0.11831285,  0.21271642, -0.07413743, ...,  0.25969714,
        -0.00562285,  0.2591503 ],
       [ 0.11554211, -0.00898063, -0.10998475, ...,  0.32584864,
         0.09761788, -0.02438263]], dtype=float32)

In [34]:
pd.DataFrame(doc_glove_vectors).to_csv("glove_embeddings_document.csv")

In [35]:
pd.DataFrame(doc_glove_vectors)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,-0.320770,0.516240,-0.352790,-0.179750,0.275050,-0.096307,0.089750,-0.279930,-0.098183,2.717800,-0.256930,-0.260300,0.430150,0.081071,-0.519580,-0.174660,-0.127410,0.235980,-0.087545,-0.288800,0.258140,0.198670,0.456920,-0.168630,-0.098067,-0.184190,-0.329630,-0.253380,0.426430,-0.372930,-0.173920,0.522640,0.103840,0.124860,0.199010,0.106560,-0.152830,0.121580,-0.129730,-0.171940,-0.153730,-0.000650,0.040749,-0.200350,0.187250,-0.083519,-0.264550,-0.149240,0.061567,0.237470,-0.403490,0.396750,-0.038913,0.046031,0.453200,-0.012772,-0.408720,-0.226730,0.019934,0.010021,-0.072433,-0.169690,-0.190660,0.349710,-0.026471,-0.242390,-0.095951,0.116740,0.430990,0.327930,-0.073179,-0.056880,0.155920,-0.037101,0.097139,0.139110,0.205370,-0.056496,-0.108870,0.347210,0.042650,-0.086676,-0.169620,-0.036511,0.064699,-0.483240,-0.144060,-0.980760,0.166140,-0.227590,-0.083246,-0.350470,-0.270100,0.072318,-0.167030,0.106780,0.114960,-0.110640,0.148230,-0.209580,-0.195060,0.299480,0.148420,0.109990,0.276380,-0.663690,0.032268,-0.080317,-0.235670,0.415660,0.088604,-0.169700,-0.067021,0.155510,0.203380,0.115610,0.081953,-0.261750,-0.062585,0.040077,0.139800,-0.448290,-0.010769,-0.210750,0.194360,0.099910,0.136350,-0.401620,0.038960,0.087963,-0.049886,0.195450,0.103370,-0.023646,0.061449,-0.076249,0.139320,-0.214160,0.086086,0.139120,-2.102600,0.289500,-0.009959,0.238360,-0.123950,-0.154790,-0.336000,0.382850,-0.338800,-0.359890,0.000357,0.117390,-0.253900,0.115600,0.143360,-0.344600,0.053722,-0.435890,-0.205810,-0.190350,-0.097652,0.099723,-0.359750,0.103090,-0.327550,0.152540,-0.039231,-0.388570,-0.066451,0.086742,-0.010071,-0.168790,0.101890,-0.674040,-0.060975,-0.042740,0.133920,-0.068049,-0.091570,-0.088889,-0.184290,-0.042847,-0.346250,-0.161550,-0.106520,-0.044709,-0.091435,-0.116410,0.055588,0.047454,0.282240,0.032870,-0.354480,-0.132740,0.131250,-0.159380,-0.112390,-0.385790,0.030209,0.201940,0.162310,0.102710,-0.099821,-0.458160,0.188560,-0.147120,-0.056286,0.068785,0.322570,0.210710,-0.130480,-0.262070,-0.163130,-0.193020,0.354830,-0.193510,-0.178520,-0.092966,-0.287570,-0.070260,0.224160,0.192530,-0.042525,-0.016779,0.153400,0.121700,-0.092059,0.098357,-0.202130,-0.311560,-0.236290,-0.062579,0.131140,-0.136930,-0.078851,0.255490,0.055127,-0.289270,-0.235590,0.450980,0.148160,-0.080504,0.343330,0.043206,-0.063912,0.108670,0.131110,-0.250220,-0.247810,0.503130,0.073038,-0.109150,-0.221500,-0.144090,-0.284910,0.286980,-0.042820,-0.020260,-0.025628,0.079759,0.393370,0.269740,0.081124,0.144930,-0.064560,0.016930,0.334950,0.168530,0.408390,0.371320,0.079057,-0.120790,-0.000983,-0.220290,0.009390,0.060437,0.017829,-0.041090,-0.039598,0.355010,-0.186480,-0.182630,-0.018606,-0.051494,-0.153260,0.024464,-0.005622,0.017035,-0.108360,0.297330,-0.225770,-0.013985,-0.294990,-0.043481,0.214270,-0.206890,-0.484670,0.184020,0.451810,0.052621
1,-0.087075,0.059598,-0.041057,-0.217988

['kickin' 'lite' 'click' ... '20hrs' '2/23/15' 'ticket(s']


In [37]:
import gensim

custom_w2v_gensim_model_path = '..\milestone1\Patrick\gensim_word2vec_google_subset.bin'
w2v_gensim_model = gensim.models.KeyedVectors.load_word2vec_format(custom_w2v_gensim_model_path, binary=True)

In [148]:
#calculates a vector for a given Tweet
def calculate_average_tweet_vector_glove(tweet, num_dimensions):
    tokens = tweet.split(' ')
    
    tweet_vector = np.zeros(num_dimensions, np.float64)
    actual_token_count = 0
    
    for token in  tokens:
        if token in glove_embed.index.values:
            #print (token)
            actual_token_count = actual_token_count + 1
            tweet_vector = np.add(tweet_vector, glove_embed.loc[glove_embed.index ==  token])

        
    tweet_vector = np.divide(tweet_vector, actual_token_count)
    
    if not tweet_vector.empty:
        tweet_vector.reset_index(drop=True, inplace=True)
    
    return tweet_vector

In [149]:
#Sanity check
calculate_average_tweet_vector_glove("arrangement flight plane", 300)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299
0,0.283233,-0.025553,0.0944,0.058597,0.15308,0.159167,-0.18097,0.21661,-0.087138,2.043333,0.029407,0.427507,0.222603,0.482623,-0.202466,-0.29957,-0.268317,1.695967,0.460157,0.018243,-0.045699,0.382463,-0.406893,0.091513,0.018113,-0.214427,-0.18362,-0.38103,0.084121,0.076007,-0.230584,0.121204,0.148416,0.532807,-0.131337,0.14309,-0.002758,-0.109175,0.041594,-0.514143,0.34761,-0.117117,0.10054,0.177198,0.330304,0.14164,-0.38156,-0.05837,0.33879,0.696357,0.080607,-0.53951,-0.218657,0.710363,-0.404313,0.164987,0.241374,0.31108,0.031467,-0.24517,0.316917,0.034447,-0.064771,-0.06762,-0.386043,0.210739,-0.274803,0.398317,-0.149977,0.065758,0.155156,-0.061325,-0.373897,0.183596,-0.411803,-0.230038,-0.324323,0.100622,0.075313,0.392463,0.088774,-0.13284,-0.022485,-0.106733,0.14077,-0.109867,0.330133,0.326698,0.266557,-0.031717,-0.172827,0.410277,-0.395793,-0.288907,-0.503377,0.149497,-0.340229,0.104933,-0.08076,0.100265,0.152359,0.10679,0.067991,0.148279,-0.336863,-1.03639,0.042801,0.118601,0.124583,0.535687,-0.24999,-0.062803,0.241293,0.48113,-0.05063,-0.234444,0.000307,0.232727,-0.192883,0.116751,-0.429423,0.118671,0.410597,-0.029,-0.103663,0.022315,0.07736,-0.004753,0.07156,-0.101909,-0.109102,-0.039467,-0.237475,-0.101158,0.386027,-0.247167,-0.335123,0.420873,-0.041473,-0.153784,-0.980233,-0.071867,-0.46258,0.025257,-0.107993,0.59009,-0.317631,-0.061969,0.523475,-0.049744,-0.488707,-0.020077,-0.663753,-0.039693,0.075775,-0.050176,0.021625,0.481317,0.308887,-0.015142,0.037606,0.02256,0.159574,-0.043993,0.29878,-0.139662,0.1311,-0.041435,0.40135,0.573127,-0.280257,0.2371,0.013839,-0.365773,0.477963,-0.089031,-0.106816,-0.214613,0.012283,-0.324304,-0.198281,-0.30238,0.029678,0.03833,0.09686,0.06523,-0.068191,-0.023573,-0.24329,0.206392,0.169187,0.306499,-0.151054,-0.356697,-0.532553,-0.179133,0.013727,-0.058043,-0.303349,0.20178,0.182122,0.13476,0.35309,0.434156,0.24808,-0.151436,-0.300886,-0.085797,0.099611,0.560646,0.288614,0.081573,0.244388,-0.137851,0.101573,0.425629,-0.09179,0.031946,0.207892,0.0677,0.343957,-0.53167,0.173959,-0.073727,0.206573,0.169545,-0.227993,0.559283,-0.137042,0.572243,0.71338,0.075733,0.077737,-0.071935,0.460553,0.04175,0.006467,-0.106937,0.017247,0.253288,0.153593,0.217033,0.2784,0.322959,-0.06236,-0.069433,-0.080783,0.16466,-0.043484,-0.160801,0.292857,0.155365,0.287398,-0.216143,0.136693,0.034258,0.331493,-0.285643,-0.014197,-0.136006,0.00314,-0.528173,-0.13966,-0.23788,0.115956,-0.384617,0.058765,0.568553,0.143187,0.250099,-0.077375,0.192547,0.30698,-0.290749,0.049723,0.545643,-0.053074,-0.315403,0.270041,0.18754,-0.091283,-0.412187,0.235206,-0.245507,0.11275,-0.002603,-0.134673,0.088785,0.12978,0.16854,-0.105044,0.158975,0.584113,-0.264511,0.159377,-0.147684,0.119625,0.523207,0.31628,0.285432


In [150]:
num_dimensions = 300
X_train['tweet_vector'] = X_train.lemmas_list.apply(lambda text: calculate_average_tweet_vector_glove(text, 300))
X_test['tweet_vector'] = X_test.lemmas_list.apply(lambda text: calculate_average_tweet_vector_glove(text, 300))

AttributeError: 'numpy.ndarray' object has no attribute 'empty'

In [41]:
df_train_tweet_vector = pd.DataFrame(list(X_train['tweet_vector']), index=X_train.index)
df_test_tweet_vector = pd.DataFrame(list(X_test['tweet_vector']), index=X_test.index)

In [42]:
#after creating new dataframes above, any zeroes are converted to null...we need to convert them back to zeroes...
df_train_tweet_vector = df_train_tweet_vector.fillna(0)
df_test_tweet_vector = df_test_tweet_vector.fillna(0)

In [43]:
X_train_combined = pd.concat([df_train_tweet_vector, X_train], axis=1)
X_test_combined = pd.concat([df_test_tweet_vector, X_test], axis=1)

### Remove `tweet_vector` and `lemmas_list` column

In [44]:
X_train_combined = X_train_combined.loc[:, X_train_combined.columns != 'tweet_vector']
X_test_combined = X_test_combined.loc[:, X_test_combined.columns != 'tweet_vector']
X_train_combined = X_train_combined.loc[:, X_train_combined.columns != 'lemmas_list']
X_test_combined = X_test_combined.loc[:, X_test_combined.columns != 'lemmas_list']

In [45]:
from sklearn.linear_model import LogisticRegression

logmodel = LogisticRegression(C=100)
logmodel.fit(X_train_combined, Y_train)
predictions = logmodel.predict(X_test_combined)

In [46]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,predictions))

              precision    recall  f1-score   support

           0       0.79      0.70      0.74      1639
           1       0.83      0.89      0.86      2753

   micro avg       0.82      0.82      0.82      4392
   macro avg       0.81      0.80      0.80      4392
weighted avg       0.82      0.82      0.82      4392



In [47]:
predictions

array([1, 1, 1, ..., 0, 1, 0])